**Use Case 1:- Precision Nutrient and Fertilizer Management** 

**Step 1:- Load & inspect the data**

In [0]:
# (Use your table name here)
df = spark.table("workspace.default.fertilizer_prediction")
df.printSchema()
df.show(5)


root
 |-- Temparature: long (nullable = true)
 |-- Humidity : long (nullable = true)
 |-- Moisture: long (nullable = true)
 |-- Soil Type: string (nullable = true)
 |-- Crop Type: string (nullable = true)
 |-- Nitrogen: long (nullable = true)
 |-- Potassium: long (nullable = true)
 |-- Phosphorous: long (nullable = true)
 |-- Fertilizer Name: string (nullable = true)

+-----------+---------+--------+---------+---------+--------+---------+-----------+---------------+
|Temparature|Humidity |Moisture|Soil Type|Crop Type|Nitrogen|Potassium|Phosphorous|Fertilizer Name|
+-----------+---------+--------+---------+---------+--------+---------+-----------+---------------+
|         26|       52|      38|    Sandy|    Maize|      37|        0|          0|           Urea|
|         29|       52|      45|    Loamy|Sugarcane|      12|        0|         36|            DAP|
|         34|       65|      62|    Black|   Cotton|       7|        9|         30|       14-35-14|
|         32|       62|      

**STEP 2:- Rename key columns for consistency**

In [0]:
# We'll standardize column names to avoid later issues.

from functools import reduce

# Current columns (case-insensitive matching)
cols_lower = {c.lower(): c for c in df.columns}

def pick(*options):
    """Pick the first existing column (case-insensitive) from options."""
    for opt in options:
        if opt.lower() in cols_lower:
            return cols_lower[opt.lower()]
    return None

# Try to find source column names commonly seen in this dataset
N_col   = pick("Nitrogen", "N")
P_col   = pick("Phosphorous", "Phosphorus", "P")
K_col   = pick("Potassium", "K")
T_col   = pick("Temperature", "Temparature", "Temp")
H_col   = pick("Humidity")
M_col   = pick("Moisture")
Soil_col= pick("Soil Type", "Soil_Type", "Soil", "SoilType")
Crop_col= pick("Crop Type", "Crop_Type", "Crop", "CropType")
Y_col   = pick("Fertilizer Name", "Fertilizer", "Fertilizer_Name", "Label", "Target")

# Build a (src -> dst) rename map for those we actually found
rename_map = {
    N_col:    "N",
    P_col:    "P",
    K_col:    "K",
    T_col:    "TEMPERATURE",
    H_col:    "HUMIDITY",
    M_col:    "MOISTURE",
    Soil_col: "SOIL_TYPE",
    Crop_col: "CROP_TYPE",
    Y_col:    "FERTILIZER"
}
rename_map = {k:v for k,v in rename_map.items() if k is not None}

# Apply renames
from pyspark.sql import DataFrame
df = reduce(lambda d, kv: d.withColumnRenamed(kv[0], kv[1]), rename_map.items(), df)

print(df.columns)
# ✅ Expected consistent names now include (whatever exists in your file):
# N, P, K, TEMPERATURE, HUMIDITY, MOISTURE, SOIL_TYPE, CROP_TYPE, FERTILIZER


['TEMPERATURE', 'Humidity ', 'MOISTURE', 'SOIL_TYPE', 'CROP_TYPE', 'N', 'K', 'P', 'FERTILIZER']


**STEP 3:- Clean and filter data**

In [0]:
# - Cast numerics
# - Drop rows with missing label
# - Filter unrealistic values to remove outliers

from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType

num_candidates = ["N","P","K","TEMPERATURE","HUMIDITY","MOISTURE"]
existing_nums = [c for c in num_candidates if c in df.columns]

for cnum in existing_nums:
    df = df.withColumn(cnum, col(cnum).cast(DoubleType()))

# Drop rows with null label
if "FERTILIZER" in df.columns:
    df = df.filter(col("FERTILIZER").isNotNull())

# Reasonable value ranges (tweak if needed; non-existing columns are ignored)
if "N" in df.columns:
    df = df.filter((col("N") >= 0) & (col("N") <= 500))
if "P" in df.columns:
    df = df.filter((col("P") >= 0) & (col("P") <= 500))
if "K" in df.columns:
    df = df.filter((col("K") >= 0) & (col("K") <= 500))
if "TEMPERATURE" in df.columns:
    df = df.filter((col("TEMPERATURE") >= -10) & (col("TEMPERATURE") <= 60))
if "HUMIDITY" in df.columns:
    df = df.filter((col("HUMIDITY") >= 0) & (col("HUMIDITY") <= 100))
if "MOISTURE" in df.columns:
    df = df.filter((col("MOISTURE") >= 0) & (col("MOISTURE") <= 100))

display(df.limit(5))


TEMPERATURE,Humidity,MOISTURE,SOIL_TYPE,CROP_TYPE,N,K,P,FERTILIZER
26.0,52,38.0,Sandy,Maize,37.0,0.0,0.0,Urea
29.0,52,45.0,Loamy,Sugarcane,12.0,0.0,36.0,DAP
34.0,65,62.0,Black,Cotton,7.0,9.0,30.0,14-35-14
32.0,62,34.0,Red,Tobacco,22.0,0.0,20.0,28-28
28.0,54,46.0,Clayey,Paddy,35.0,0.0,0.0,Urea


**STEP 4:- Feature engineering**

In [0]:
# Add a couple of useful composite features (keeps your flow consistent)
from pyspark.sql.functions import when, lit

if all(c in df.columns for c in ["N","P","K"]):
    df = df.withColumn("NPK_RATIO", (col("N") + lit(1.0)) / (col("P") + col("K") + lit(2.0)))

if all(c in df.columns for c in ["TEMPERATURE","HUMIDITY"]):
    df = df.withColumn("TEMP_HUMID_INDEX", col("TEMPERATURE") * (col("HUMIDITY")/100.0))

display(df.select([c for c in ["FERTILIZER","N","P","K","NPK_RATIO","TEMP_HUMID_INDEX"] if c in df.columns]).limit(5))


FERTILIZER,N,P,K,NPK_RATIO
Urea,37.0,0.0,0.0,19.0
DAP,12.0,36.0,0.0,0.34210526315789475
14-35-14,7.0,30.0,9.0,0.1951219512195122
28-28,22.0,20.0,0.0,1.0454545454545454
Urea,35.0,0.0,0.0,18.0


**STEP 5:- SQL exploratory analysis**

In [0]:
# Step 5. SQL exploratory analysis
df.createOrReplaceTempView("fert")

# Average N/P/K by crop and top fertilizers by frequency
spark.sql("""
  SELECT
    COALESCE(CROP_TYPE, 'UNKNOWN') AS CROP_TYPE,
    COUNT(*) AS total_records,
    ROUND(AVG(N),2) AS avg_N,
    ROUND(AVG(P),2) AS avg_P,
    ROUND(AVG(K),2) AS avg_K
  FROM fert
  GROUP BY CROP_TYPE
  ORDER BY total_records DESC
""").show(10)

spark.sql("""
  SELECT FERTILIZER, COUNT(*) AS cnt
  FROM fert
  GROUP BY FERTILIZER
  ORDER BY cnt DESC
""").show(10)
# ✅ Screenshot these for your PPT.


+-----------+-------------+-----+-----+-----+
|  CROP_TYPE|total_records|avg_N|avg_P|avg_K|
+-----------+-------------+-----+-----+-----+
|  Sugarcane|           13|14.62|17.62| 4.23|
|     Cotton|           12|16.42|23.83| 3.33|
|    Millets|           11|23.18|14.36|  0.0|
|      Paddy|           10| 20.8| 16.3|  3.7|
|     Pulses|           10| 18.4| 17.5|  4.2|
|      Wheat|            9|23.33|14.44| 2.89|
|     Barley|            7|12.57| 26.0| 5.29|
|  Oil seeds|            7| 19.0|17.29| 2.29|
|    Tobacco|            7|19.14|19.29| 4.86|
|Ground Nuts|            7|23.29|21.57|  2.0|
+-----------+-------------+-----+-----+-----+
only showing top 10 rows
+----------+---+
|FERTILIZER|cnt|
+----------+---+
|      Urea| 22|
|       DAP| 18|
|     28-28| 17|
|     20-20| 14|
|  14-35-14| 14|
|  17-17-17|  7|
|  10-26-26|  7|
+----------+---+



**STEP 6:- Prepare ML features**

In [0]:
# We'll predict FERTILIZER (multi-class). Categorical: SOIL_TYPE, CROP_TYPE

from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline

label_col = "FERTILIZER"
cat_cols = [c for c in ["SOIL_TYPE","CROP_TYPE"] if c in df.columns]
num_cols = [c for c in ["N","P","K","TEMPERATURE","HUMIDITY","MOISTURE","NPK_RATIO","TEMP_HUMID_INDEX"] if c in df.columns]

# Label indexer
label_indexer = StringIndexer(inputCol=label_col, outputCol="label", handleInvalid="keep")

# Categorical indexers + OHE
indexers = [StringIndexer(inputCol=c, outputCol=f"{c}_idx", handleInvalid="keep") for c in cat_cols]
encoders = [OneHotEncoder(inputCols=[f"{c}_idx"], outputCols=[f"{c}_ohe"]) for c in cat_cols]

ohe_cols = [f"{c}_ohe" for c in cat_cols]
assembler = VectorAssembler(
    inputCols=num_cols + ohe_cols,
    outputCol="features"
)
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")


**STEP 7:- Model training (Logistic Regression - multinomial)**

In [0]:
# Step 7. Model training (Logistic Regression - multinomial)
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol="scaledFeatures", labelCol="label", maxIter=100, regParam=0.0, elasticNetParam=0.0)

stages = [label_indexer] + indexers + encoders + [assembler, scaler, lr]
pipeline = Pipeline(stages=stages)

train, test = df.randomSplit([0.8, 0.2], seed=42)
model = pipeline.fit(train)
pred = model.transform(test)


**STEP 8:- Model evaluation**

In [0]:
# 🟢 Step 8. Model evaluation
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy").evaluate(pred)
f1  = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1").evaluate(pred)
print("Accuracy:", round(acc, 3))
print("F1 Score:", round(f1, 3))

pred.createOrReplaceTempView("predictions")
spark.sql("""
SELECT prediction AS predicted_label_idx, COUNT(*) AS count
FROM predictions
GROUP BY prediction
ORDER BY count DESC
""").show()
# ✅ Index -> class mapping (for your PPT):
label_map = {i: lab for i, lab in enumerate(model.stages[0].labels)}
print("Label index mapping:", label_map)


Accuracy: 0.87
F1 Score: 0.845
+-------------------+-----+
|predicted_label_idx|count|
+-------------------+-----+
|                4.0|    6|
|                0.0|    5|
|                2.0|    4|
|                3.0|    4|
|                6.0|    3|
|                1.0|    1|
+-------------------+-----+

Label index mapping: {0: 'Urea', 1: 'DAP', 2: '28-28', 3: '14-35-14', 4: '20-20', 5: '17-17-17', 6: '10-26-26'}


**STEP 9:- Save model and results**

In [0]:
# Save predictions as a SQL table for downstream SQL/visuals
cols_to_keep = [c for c in ["CROP_TYPE","SOIL_TYPE","N","P","K","TEMPERATURE","HUMIDITY","MOISTURE","FERTILIZER"] if c in pred.columns]
save_df = pred.select(*(cols_to_keep + ["prediction","probability"]))

save_df.write.mode("overwrite").saveAsTable("fertilizer_predictions")
print("✅ Predictions saved as SQL table: fertilizer_predictions")

# Example: most-likely fertilizer per crop (using predicted index -> name)
spark.udf.register("idx_to_label", lambda i: label_map.get(int(i), "UNKNOWN"))
spark.sql("""
SELECT
  COALESCE(CROP_TYPE, 'UNKNOWN') AS CROP_TYPE,
  idx_to_label(first(prediction)) AS predicted_top_fertilizer,
  COUNT(*) AS records
FROM fertilizer_predictions
GROUP BY CROP_TYPE
ORDER BY records DESC
""").show(15)


✅ Predictions saved as SQL table: fertilizer_predictions
+---------+------------------------+-------+
|CROP_TYPE|predicted_top_fertilizer|records|
+---------+------------------------+-------+
|   Pulses|                   20-20|      4|
|Sugarcane|                   20-20|      4|
|    Paddy|                    Urea|      2|
|  Tobacco|                10-26-26|      2|
|Oil seeds|                   20-20|      2|
|  Millets|                    Urea|      2|
|    Maize|                14-35-14|      2|
|    Wheat|                    Urea|      2|
|   Barley|                14-35-14|      2|
|   Cotton|                   28-28|      1|
+---------+------------------------+-------+



**STEP 10:- Run the SQL and store it as a DataFrame to display as a Table**

In [0]:
top_fert = spark.sql("""
SELECT
    COALESCE(CROP_TYPE, 'UNKNOWN') AS CROP_TYPE,
    idx_to_label(first(prediction)) AS predicted_top_fertilizer,
    COUNT(*) AS records
FROM fertilizer_predictions
GROUP BY CROP_TYPE
ORDER BY records DESC
""")
display(top_fert)




CROP_TYPE,predicted_top_fertilizer,records
Pulses,20-20,4
Sugarcane,20-20,4
Paddy,Urea,2
Tobacco,10-26-26,2
Oil seeds,20-20,2
Millets,Urea,2
Maize,14-35-14,2
Wheat,Urea,2
Barley,14-35-14,2
Cotton,28-28,1


Databricks visualization. Run in Databricks to view.